In [29]:
# 모듈 임포트
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle # pickle을 이용하면 int 값을 그대로 저장하고 그대로 불러옴
from konlpy.tag import Okt # 형태소 분리기 임포트

In [30]:
# 판다스 줄맞춤 옵션 설정 (카테고리, 타이틀 맞춤)
pd.set_option('display.unicode.east_asian_width', True)

In [31]:
# 파일 읽어오기
df = pd.read_csv('/content/datasets/naver_news_titles_210616.csv', index_col=0)

print(df.head())
print(df.info())

                                                                title  category
0    김총리  새 거리두기 방안  일상에 큰 변화 될 것                    Politics
1     백신 유급휴가비  지원한다 법개정안 복지위 통과                   Politics
2   쇄신   변화  강조한 송영길  개혁 속도조절  국민공감대 우선 ......  Politics
3    소통수석  열린자세로  의 중요 이웃국 역할할 것  종합 ......       Politics
4             큐어백  백신 도입     한국  생산 거점                    Politics
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28309 entries, 0 to 28308
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     28309 non-null  object
 1   category  28309 non-null  object
dtypes: object(2)
memory usage: 663.5+ KB
None


In [32]:
# 중복 제거
col_dup = df['title'].duplicated()
print(col_dup)

sum_dup = df.title.duplicated().sum()
print(sum_dup)

0        False
1        False
2        False
3        False
4        False
         ...  
28304    False
28305    False
28306    False
28307    False
28308    False
Name: title, Length: 28309, dtype: bool
1234


In [33]:
# title 컬럼 기준으로 중복 제거 (row 통째 제거)
df = df.drop_duplicates(subset=['title'])

sum_dup = df.title.duplicated().sum()
print(sum_dup)

0


In [34]:
# 인덱스 새로 고침
df.reset_index(drop=True, # False로 줄 시 기존 인덱스를 colum으로 올림
               inplace=True)

In [35]:
# 데이터 나누기
X = df['title']
Y = df['category']

In [36]:
# 원핫인코딩 위해 라벨로 변경
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_

print(label)

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']


In [37]:
# 지정한 라벨 저장
# 이미 학습한 라벨들을 이용해야 학습된 결과들과 똑같이 도출

with open('/content/datasets/category_encoder.pickle', 'wb') as f:
    pickle.dump(encoder, f)   

In [38]:
print(labeled_Y)

[3 3 3 ... 2 2 2]


In [39]:
# 원핫인코딩
onehot_Y = to_categorical(labeled_Y)

print(onehot_Y)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [40]:
# 한국어 형태소 분리기 설치
# 앞에 !를 쓰면 터미널에서 동작하는 코드로 인식
# !pip install konlpy

In [41]:
# 첫번째 데이터 형태소로 분리
okt = Okt()
print(type(X))

okt_X = okt.morphs(X[0])
print(X[0])
print(okt_X)

<class 'pandas.core.series.Series'>
김총리  새 거리두기 방안  일상에 큰 변화 될 것 
['김', '총리', '새', '거리', '두기', '방안', '일상', '에', '큰', '변화', '될', '것']


In [42]:
# 모든 row를 형태소로 분리
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0        [김, 총리, 새, 거리, 두기, 방안, 일상, 에, 큰, 변화, 될, 것]...
1        [백신, 유급, 휴가, 비, 지원, 한, 다, 법, 개정안, 복지, 위, 통과]...
2        [쇄신, 변화, 강조, 한, 송영길, 개혁, 속도, 조절, 국민, 공감, 대, 우선...
3        [소통, 수석, 열린, 자세, 로, 의, 중요, 이웃, 국, 역할, 할, 것, 종합...
4                 [큐어, 백, 백신, 도입, 한국, 생산, 거점]
                               ...                        
27070    [2만, 4000년, 간, 죽지, 않은, 좀비, 가, 시베리아, 에서, 나타났다]...
27071    [PLAY, IT, 갤럭시, 북, 경험, 키, 보드, 에서도, 블루투스, 키, 보드...
27072    [PLAY, IT, 카카오, 판, 클럽, 하우스, 음, 체험, 기, 작, 지만, 큰...
27073    [PLAY, IT, 홈쇼핑, 고객, 정착, 가능할까, CJ, 온스타일, 이용, 해보...
27074    [PLAY, IT, 크롬, 잡겠다는, 네이버, 웨일, 신, 기능, 사용, 해보니]...
Name: title, Length: 27075, dtype: object


In [43]:
# stopwords 제거를 위해 불러오기
stopwords = pd.read_csv('/content/datasets/stopwords.csv')

In [44]:
print(stopwords.head(30))

    Unnamed: 0  stopword
0            0        아
1            1        휴
2            2    아이구
3            3    아이쿠
4            4    아이고
5            5        어
6            6        나
7            7      우리
8            8      저희
9            9      따라
10          10      의해
11          11        을
12          12        를
13          13        에
14          14        의
15          15        가
16          16      으로
17          17        로
18          18      에게
19          19    뿐이다
20          20  의거하여
21          21  근거하여
22          22  입각하여
23          23  기준으로
24          24    예하면
25          25      예를
26          26      들면
27          27    들자면
28          28        저
29          29      소인


In [45]:
# stopwords 제거
words = [] # 빈 리스트 생성
for word in okt_X: # stopwords에 없는 단어들만 리스트에 담기
    if word not in list(stopwords['stopword']):
        words.append(word)
print(words)

['김', '총리', '새', '거리', '두기', '방안', '일상', '큰', '변화', '될']


In [46]:
# 의미 없는 형태소 제외
for i in range(len(X)): # 모든 문장을 첫번째 요소부터 찾기
    result = []
    for j in range(len(X[i])): # i번째 문장의 첫번째 요소부터 찾기
        if len(X[i][j]) > 1: # 한글자인 단어들은 if문 안에 못들어간다
            if X[i][j] not in list(stopwords['stopword']): # stopwords에 없는 단어 리스트에 추가
                result.append(X[i][j]) # 통과된 텍스트를 빈 리스트에 넣어주기
    X[i] = ' '.join(result) # 리스트 요소들을 join으로 이어준다

print(X)

0                            총리 거리 두기 방안 일상 변화
1                     백신 유급 휴가 지원 개정안 복지 통과
2           쇄신 변화 강조 송영길 개혁 속도 조절 국민 공감
3                  소통 수석 열린 자세 중요 이웃 역할 종합
4                            큐어 백신 도입 한국 생산 거점
                               ...                        
27070          2만 4000년 죽지 않은 좀비 시베리아 나타났다
27071    PLAY IT 갤럭시 경험 보드 에서도 블루투스 보드 트리오 500 보니...
27072                 PLAY IT 카카오 클럽 하우스 체험 차이
27073    PLAY IT 홈쇼핑 고객 정착 가능할까 CJ 온스타일 이용 해보니...
27074    PLAY IT 크롬 잡겠다는 네이버 웨일 기능 사용 해보니...
Name: title, Length: 27075, dtype: object


In [47]:
# 라벨레코딩 (단어들을 라벨링)
 
token = Tokenizer() # 토크나이저 객체 생성
token.fit_on_texts(X) # 어떤 형태소를 몇 번으로 바꿀 지 지정
tokened_X = token.texts_to_sequences(X) # 시퀀스화

# 라벨링 확인
print(tokened_X[0])

[281, 74, 138, 873, 603, 126]


In [48]:
# 지정한 토큰 저장
# 새로운 헤드라인을 토큰화할 때 기존 토큰들이 필요 => 이미 학습한 토큰들을 이용해야 학습된 결과들과 똑같이 도출

with open('/content/datasets/news_token.pickle', 'wb') as f:
    pickle.dump(token, f) # du9mp 함수를 이용하여 token을 f에 저장

In [49]:
wordsize = len(token.word_index) + 1 # 각 단어 별 붙여진 번호를 볼 수 있다, len을 써서 붙인 단어의 개수를 출력함(0이 없어서 0을 추가하기 위해서 1을 더해준다

print(wordsize)

24151


In [50]:
#문장별 형태소 갯수가 달라서 맞춰주기

max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):  # 문장을 하나하나 꺼내서 길이를 보면서 가장 큰 문장을 찾아준다
        max = len(tokened_X[i])  # 가장 긴 문장을 다시 max값으로 지정

print(max)  #최대 문장 길이= 16

27


In [51]:
X_pad = pad_sequences(tokened_X, max) # 토큰의 갯수가 16개가 되게 앞쪽에 0을 붙여줌
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   281    74   138
    873   603   126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     2  4851  1001   105
   1953  2506   189]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  2507   126  1030    66   636   219
   2825     7  1245]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   923  1803  1804  3229  1421
   1547  1095     4]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0  1805     2   312
     11   156  2661]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    66    19  1139 13780  3725  1139
   1059

In [52]:
# train, test set 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size = 0.1)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [53]:
xy = X_train, X_test, Y_train, Y_test
np.save('/content/datasets/news_data_max_{}_size_{}'.format(max, wordsize), xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
